In [10]:
#import spotifycharts as sc
#from spotifycharts import settings
#from spotifycharts import classes
import requests
from datetime import date, timedelta
from time import sleep
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd

In [11]:
countries = {'ar':'Argentina', 'at':'Austria', 'au':'Australia', 'be':'Belgium',
       'bo':'Bolivia', 'br':'Brazil', 'ca':'Canada', 'ch':'Switzerland', 
       'cl':'Chile', 'co':'Columbia', 'cr':'CostaRica', 'cz':'CzechRepublic',
       'de':'Germany', 'dk':'Denmark', 'do':'DominicanRepublic', 'ec':'Ecuador', 
       'ee':'Estonia', 'es':'Spain', 'fi':'Finland', 'fr':'France', 
       'gb':'UnitedKingdom', 'global':'World', 'gr':'Greece', 'gt':'Guatemala',
       'hk':'HongKong', 'hn':'Honduras', 'hu':'Hungary', 'id':'Indonesia', 
       'ie':'Ireland', 'is':'Iceland', 'it':'Italy', 'jp':'Japan', 
       'lt':'Lithuania', 'lu':'Luxemborg', 'lv':'Latvia', 'mx':'Mexico', 
       'my':'Malaysia', 'nl':'Netherlands', 'no':'Norway', 'nz':'NewZealand', 
       'pa':'Panama', 'pe':'Peru', 'ph':'Philippines', 'pl':'Poland', 
       'pt':'Portugal', 'py':'Paraguay', 'se':'Sweden', 'sg':'Singapore', 
       'sk':'Slovakia', 'sv':'ElSalvador', 'tr':'Turkey', 'tw':'Taiwan', 'us':'USA', 'uy':'Uruguay',} 

In [12]:
country_codes=list(countries.keys())

In [13]:
country_codes

['ar',
 'at',
 'au',
 'be',
 'bo',
 'br',
 'ca',
 'ch',
 'cl',
 'co',
 'cr',
 'cz',
 'de',
 'dk',
 'do',
 'ec',
 'ee',
 'es',
 'fi',
 'fr',
 'gb',
 'global',
 'gr',
 'gt',
 'hk',
 'hn',
 'hu',
 'id',
 'ie',
 'is',
 'it',
 'jp',
 'lt',
 'lu',
 'lv',
 'mx',
 'my',
 'nl',
 'no',
 'nz',
 'pa',
 'pe',
 'ph',
 'pl',
 'pt',
 'py',
 'se',
 'sg',
 'sk',
 'sv',
 'tr',
 'tw',
 'us',
 'uy']

In [14]:
processed_countries = []
processed_countries

[]

In [15]:
country_codes = [x for x in country_codes if x not in processed_countries]

In [16]:
country_codes

['ar',
 'at',
 'au',
 'be',
 'bo',
 'br',
 'ca',
 'ch',
 'cl',
 'co',
 'cr',
 'cz',
 'de',
 'dk',
 'do',
 'ec',
 'ee',
 'es',
 'fi',
 'fr',
 'gb',
 'global',
 'gr',
 'gt',
 'hk',
 'hn',
 'hu',
 'id',
 'ie',
 'is',
 'it',
 'jp',
 'lt',
 'lu',
 'lv',
 'mx',
 'my',
 'nl',
 'no',
 'nz',
 'pa',
 'pe',
 'ph',
 'pl',
 'pt',
 'py',
 'se',
 'sg',
 'sk',
 'sv',
 'tr',
 'tw',
 'us',
 'uy']

In [17]:
def add_url(dates, base_url):
    url_list=[]
    for date in dates:
        c_string = base_url+date
        url_list.append(c_string)
    return url_list

def song_scrape(x,songs,country):
    pg = x
    pos=1
    song_lst = []
    for tr in songs.find("tbody").findAll("tr"):
        artist= tr.find("td", {"class": "chart-table-track"}).find("span").text
        artist= artist.replace("by ","").strip()
  
        title= tr.find("td", {"class": "chart-table-track"}).find("strong").text
    
        streams = tr.find("td", {"class": "chart-table-streams"}).text
        streams = int(streams.replace(',',''))
 
        songid= tr.find("td", {"class": "chart-table-image"}).find("a").get("href")
        #songid= songid.split("track/")[1]
        songid_lst = songid.split("track/")
        songid = ''
        if len(songid_lst)>=2:
            songid = songid_lst[1]
    
        url_date= x.split("daily/")[1]
        song_lst.append([pos, title, artist, streams, songid, url_date, country])
        pos+=1
    return song_lst
	
#loop through urls to create array of all of our song info


def get_songs(countries, start_date, end_date):
    
    for country in tqdm(countries):
        final = []

        url = f"https://spotifycharts.com/regional/{country}/daily/"

        delta= end_date-start_date
        dates_=[]

        for i in range(delta.days+1):
            day = start_date+timedelta(days=i)
            day_string= day.strftime("%Y-%m-%d")
            dates_.append(day_string)

        url_list_=add_url(dates_,url)

        for u in tqdm(url_list_, leave=False):
            #print(u)

            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
            read_pg= requests.get(u,headers=headers)
            sleep(2)
            #print(read_pg.text)
            soup= BeautifulSoup(read_pg.text, "html.parser")
            songs_= soup.find("table", {"class":"chart-table"})
            #print(songs)
            if songs_ is not None:
                song_lst= song_scrape(u,songs_,country)
                #print(song_lst)
                final= final + song_lst
            else:
                print(f"No chart for {u}")

        #convert to data frame with pandas for easier data manipulation

        final_df = pd.DataFrame(final, columns= ["Position", "Track Name", "Artist", "Streams", "URL", "Date", "Region"])

        #write to csv

        with open(f'sp_data_{country}.csv', 'w', encoding="utf-8") as f:
                final_df.to_csv(f, header= True, index=False)

In [18]:
start_date= date(2021, 6, 6)
end_date= date(2021, 11, 24)
get_songs(country_codes, start_date, end_date)

  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/172 [00:00<?, ?it/s]

  0%|          | 0/172 [00:00<?, ?it/s]

  0%|          | 0/172 [00:00<?, ?it/s]

  0%|          | 0/172 [00:00<?, ?it/s]

...

  0%|          | 0/172 [00:00<?, ?it/s]

No chart for https://spotifycharts.com/regional/tr/daily/2021-06-28


  0%|          | 0/172 [00:00<?, ?it/s]

  0%|          | 0/172 [00:00<?, ?it/s]

  0%|          | 0/172 [00:00<?, ?it/s]

In [19]:
print("That's all folks!!")

That's all folks!!


In [ ]:
name = classes.Name('top200')
name_code = settings.NAME_CODES[name]
name_code

In [ ]:
url = f'{settings.SPOTIFY_CHARTS_URL}/{name_code}/'
url

In [ ]:
with requests.Session() as session:
    response = session.get(url)

In [ ]:
response

In [ ]:
regions={}
if response.status_code != requests.codes.OK:
    print(f'{settings.LOG_REGIONS_DOWNLOAD_WARNING} of {name} charts')
    print(regions)
parser = bs4.BeautifulSoup(response.text,'html.parser')

In [ ]:
regions = sc.download_regions('r')
regions

In [ ]:
chart_downloader = sc.ChartDownloader(name='viral50',
                                      periodicity='weekly',
                                      region_names=['germany', 'france'],
                                      begin_date='2019-12-30',
                                      end_date='2020-01-05',
                                      cpu_count=2,
                                      directory_path='data')

charts = {}
for region_name in chart_downloader.region_names:
    charts[region_name] = chart_downloader[region_name]